In [1]:
import cv2
import pytesseract as tess
import numpy as np
from pdf2image import convert_from_path
from PIL import Image
import re
import pandas as pd
# help(tess)
tess.pytesseract.tesseract_cmd  = r'C:\Users\Pfactorial00110\anaconda3\Library\bin\tesseract'
#poppler_path = r'C:\Users\Pfactorial00110\Downloads\poppler-0.68.0_x86\poppler-0.68.0\bin'

In [2]:
def ocr_pdf(pdf):

    images = convert_from_path(pdf)
    img = cv2.cvtColor(np.array(images[0]), cv2.COLOR_RGB2BGR)
    content=tess.image_to_string(img)
    return content

In [8]:
images = convert_from_path('02162022.pdf')
im_list = [im1.crop((600,50,im1.size[0]-50,im1.size[1]-50)) for im1 in images]

In [564]:
full_text = []
for i,ims in enumerate(im_list):
    con = tess.image_to_string(ims)
    full_text.append(con)

# page 6

In [506]:
def sales_sold():
    sales = [i.replace('\n','***') for i in full_text[6:8]][0].split("ITEM CATEGORIES SOLD")[0]
    sales_1 = sales.split('***')
    sales_1 = [i.replace(',','') for i in sales_1]
    # colnames = sales_summ_1[1].split(' ')
    sold = pd.DataFrame()
    for i in range(len(sales_1)):
        try:
            Type = re.findall('.+\D',sales_1[i])[0]#[:-1]
            try:
                price = re.findall('[0-9]+',sales_1[i])[-1]
            except:
                price = " "
            # print(ename,gross,net)
            sold = sold.append(pd.Series([Type,price]).T, ignore_index=True)
        except:
            pass
    sold.columns = ['Type','price']
    return sold

In [565]:
sale = sales_sold()
sale

,Type,price
0,CLASSIC THIN CRUST,
1,Medium,
2,COLD SUBS,
3,Half,1
4,CRUSTS,
5,"10"" GLUTEN FREE",2
6,Large,40
7,Medium,6
8,Small,8
9,"Solo(8"")",2


# page 7

In [181]:
def order_duration_summary():
    order_1 = [i.replace('\n','***') for i in full_text[6:8]][0].split("LABOR COSTS")[0]
    order_2 = [i.replace('\n','***') for i in full_text[6:8]][1].split("LABOR COSTS")[0]
    order = str(re.findall(r"BY ORDER.+",order_1))
    orders = order+order_2
    orders_1 = orders.split('***')
    orders_1 = [i.replace(',','') for i in orders_1]
    summary = pd.DataFrame()
    for i in range(len(orders_1)):
        try:
            Type = re.findall('.+?\d',orders_1[i])[0][:-2]
            try:
                orders = re.findall('\d+\s',orders_1[i])[0]
            except:
                orders =  ' '
            Time = re.findall('\d+[.]\d+',orders_1[i])[0]
            # print(ename,gross,net)
            summary = summary.append(pd.Series([Type,orders,Time]).T, ignore_index=True)
        except:
            pass
    summary.columns = ['Type','orders_1','Avg Time']
    return summary

In [183]:
summary = order_duration_summary()
summary

,Type,orders_1,Avg Time
0,Delivery,18,2.69
1,Pick-up,20,0.76
2,Walk-In,87,0.41
3,BOIES NATE,6,1.18
4,BOTHUN ANTHONY,6,0.93
5,CARNEY JACK,14,0.88
6,EASTLUND ANTHONY,81,0.46
7,FLAXMAN MARK,3,0.95
8,JENSEN IAN,12,2.34
9,ZANTEK ANDREA Z,,2.15
